<a href="https://colab.research.google.com/github/plaban1981/POCs/blob/main/Question_generation_geography__using_Ca_questgen_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PyPDF

In [3]:
!pip install install PyPDF2

     |████████████████████████████████| 77 kB 3.2 MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61101 sha256=1b44476da5f48eb35e5d24f38ea5ba269b9187827f63f14b7ec895c4a3155d72
  Stored in directory: /root/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2


## Questgen Api

In [68]:
%%capture
!pip install git+https://github.com/ramsrigouthamg/Questgen.ai

In [69]:
!pip install --quiet git+https://github.com/boudinfl/pke.git

In [71]:
%%capture
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz

In [72]:
!tar -xvf  s2v_reddit_2015_md.tar.gz

./._s2v_old
./s2v_old/
./s2v_old/._freqs.json
./s2v_old/freqs.json
./s2v_old/._vectors
./s2v_old/vectors
./s2v_old/._cfg
./s2v_old/cfg
./s2v_old/._strings.json
./s2v_old/strings.json
./s2v_old/._key2row
./s2v_old/key2row


In [73]:
!ls s2v_old

cfg  freqs.json  key2row  strings.json	vectors


In [74]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Download sense2vec in order to generate multiple choices

In [70]:
%%capture
!python -m nltk.downloader universal_tagset
!python -m spacy download en

## Import Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Extract the number of pages in PDF file 

In [30]:
from PyPDF2 import PdfFileReader
path = '/content/drive/MyDrive/ZeoanAI_Poc/geography_tutorial.pdf'
def get_pdf_page_count(path):
  with open(path, 'rb') as fl:
    reader = PdfFileReader(fl)
    return reader.getNumPages()

In [5]:
get_pdf_page_count(path)

167

In [27]:
file=open(path,"rb") 
reader=PdfFileReader(file)
page1=reader.getPage(7)
pdfData=page1.extractText()
print(type(pdfData))
print(pdfData.replace(".","").strip().replace("\n",""))


<class 'str'>
Geography 7    India is located entirely in the northern hemisphere; specifically in the south-central part of the continent of Asia    The mainland of India extends between latitudes 80and 370 and longitudes 680and 970 (as shown in the map given below) The southern boundary extends up to 6°45' N latitude in the Bay of Bengal    With an area of 328 million square km, India is the 7th largest country of the world   The six largest countries of the world in decreasing order are Russia, Canada, USA, China, Brazil, and Australia    India accounts for about 24 percent of the total geographical area of the world   India has a total land boundary of about 15,200 km  1 INDIA - Introduction


In [31]:
num_pages = get_pdf_page_count(path)

## Extract the Table of Contents

In [32]:
pages = '1-'+str(num_pages)
pages

'1-167'

In [34]:
def read_pdf(path,page_num):
  with open(path, "rb") as filehandle:
    pdf = PdfFileReader(filehandle)
    page1 = pdf.getPage(page_num)
    content = page1.extractText()
    return content

In [36]:
read_pdf(path,7)

"Geography\n \n7\n \n \n\n\n\n\n \n\n \nIndia is located entirely in the \nnorthern hemisphere\n; specifically\n \nin the \nsouth\n-\ncentral part of the continent of Asia.\n \n \n \n\n \nThe main\nland of India extends between latitudes \n8\n0\n\nand\n \n37\n0\n\n \nand \nlongitudes \n68\n0\n\nand\n \n97\n0\n\n \n(as shown in the map given\n \nbelow)\n.\n \nT\nhe \nsouthern boundary extends up to \n6°45' N\n \nlatitude in the Bay of Bengal.\n \n \n\n\n\n\n \n\n \nWith an area \nof 3.28 million square km\n, India is \nthe \n7\nth\n \nlargest country of the \nworld.\n \n \n\n \nThe \nsix\n \nlargest countries of the world in decreasing order are Russia,\n \nCanada, \nUSA, China, Brazil, and Australia. \n \n \n\n \nIndia accounts\n \nfor about\n \n2.4 percent\n \nof the total geographical area of the \nworld\n.\n \n \n\n \nIndia has \na \ntotal land boundary \nof about\n \n15,200 km\n.\n \n \n1.\n \nINDIA \n-\n \nIntroduction\n \n"

## Define helper functions

1. get number of pages in the pdf file

In [37]:
def get_pdf_page_count(path):
  with open(path, 'rb') as fl:
    reader = PdfFileReader(fl)
    return reader.getNumPages()

2. Read the content of the pdf that has no tables

In [38]:
def read_pdf(path,page_num):
  with open(path, "rb") as filehandle:
    pdf = PdfFileReader(filehandle)
    page1 = pdf.getPage(page_num)
    content = page1.extractText()
    return content

3. function to remove html tag if any

In [39]:
from bs4 import BeautifulSoup
def remove_tag(text):
  cleantext = BeautifulSoup(text, "lxml").text
  return cleantext

4. function to remove punctuations

In [40]:
import string,re
def remove_punc(text):
  #clean_text = " ".join([word for word in text.split() if word not in string.punctuation])
  #clean_text = text.translate(str.maketrans('', '', string.punctuation))
  clean_text = re.sub(r"[-()\"#/@;:<>{}`+=~|!?,]", "", text)
  return clean_text

5. function to clean special characters in the pdf

In [61]:
def clean_pdf_text(path,page_num):
  pdf_text = read_pdf(path,page_num)
  clean_text = remove_tag(pdf_text)
  clean_text = remove_punc(clean_text)
  clean_text = clean_text.replace("\n","")
  return clean_text[1:].rstrip()


In [65]:
path = "/content/drive/MyDrive/ZeoanAI_Poc/CRR_SRR.pdf"
num_pages = get_pdf_page_count(path)
print(num_pages)
document_list = []
for i in range (2,num_pages):
  document_list.append(clean_pdf_text(path,i))


14


In [66]:
document_list

['Master Circular    Cash  Reserve Ratio CRR andStatutory Liquidity Ratio SLR1. GeneralWith a view to monitoring compliance with statutory reserve requirements viz. Cash Reserve Ratioand Statutory Liquidity Ratio by the Scheduled Commercial Banks Reserve Bank of India hasprescribed statutory returns i.e. Form A return for CRR under Section 42 2 of the RBI Act 1934and Form VIII return for SLR under Section 24 of the Banking Regulation Act 1949. The broaddetails of the reserve requirements are summarised below.2.  Cash Reserve Ratio CRR2.1 Maintenance of CRRIn terms of Section 421 of the RBI Act 1934 Scheduled Commercial Banks are required tomaintain with RBI an average cash balance the amount of which shall not be less than three percent of the total of the Net Demand and Time Liabilities NDTL in India on a fortnightly basis andRBI is empowered to increase the said rate of CRR to such higher rate not exceeding twentypercent of the Net Demand and Time Liabilities NDTL under the RBI Act 1

## Main Function 

Function to check if table exists in a pdf, if exists check for tables.
If table found store the details into dcuments list else store it in a no_pd_list

In [41]:
def generate_questions(question_list):
  mcq_questions_list = []
  short_questions_list = []
  boolean_questions_list = []
  for question in question_list:
    print(f'context : {question}')
    print("\n")
    # remove html tag
    print("Remove HTML tags ......")
    print("\n")
    clean_text = remove_tag(question)
    print(f'cleaned_text : {clean_text}')
    print("\n")
    # remove punctuations
    print('Remove punctuations from the text........')
    print("\n")
    clean_text = remove_punc(clean_text)
    print(f"cleaned texts :{clean_text}")
    print("\n")
    # generate payload
    print('Genearting payload ................')
    print("\n")
    payload = {"input_text":clean_text}
    # boolean questions
    print("Generating boolean questions ......")
    print("\n")
    boolean_questions_list.append(generate_boolq_list(payload))
    # short questions
    print('Generating short questions .........')
    print("\n")
    short_questions_list.append(generate_shortq_list(payload))
    # multiple choice question
    print('Generating Multiple Choice Questions .....')
    print("\n")
    mcq_questions_list.append(generate_mcq_list(payload))
  return mcq_questions_list,short_questions_list,boolean_questions_list

## Invoke the main function

In [75]:
import pandas as pd
from pprint import pprint
from Questgen import main

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unz

## Generate MCQ Questions

In [77]:
qg = main.QGen()

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [82]:
qe= main.BoolQGen()

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

## Function to generate multiple choice questions

In [78]:
def generate_mcq_list(text):
  output = qg.predict_mcq(text)
  mcq_questions_list = []
  if len(output) > 0:
    for items in output['questions']:
      mcq_questions_list.append(items['question_statement'])
  return mcq_questions_list

## Function to generate short questions

In [79]:
def generate_shortq_list(text):
  output = qg.predict_shortq(text)
  short_questions_list = []
  if len(output) > 0:
    for items in output['questions']:
      short_questions_list.append(items['Question'])
  return short_questions_list

## Function to generate boolean questions

In [80]:
def generate_boolq_list(text):
  output = qe.predict_boolq(text)
  if len(output) > 0 :
    boolean_questions_list = output['Boolean Questions']
  else:
    boolean_questions_list = []

  return boolean_questions_list

In [83]:
import time
start = time.time()
%timeit
mcq_questions_list,short_questions_list,boolean_questions_list = generate_questions(document_list)
end  = time.time()
print(f"Time taken to complete the entire task in seconds : {end - start}")

context : Master Circular    Cash  Reserve Ratio CRR andStatutory Liquidity Ratio SLR1. GeneralWith a view to monitoring compliance with statutory reserve requirements viz. Cash Reserve Ratioand Statutory Liquidity Ratio by the Scheduled Commercial Banks Reserve Bank of India hasprescribed statutory returns i.e. Form A return for CRR under Section 42 2 of the RBI Act 1934and Form VIII return for SLR under Section 24 of the Banking Regulation Act 1949. The broaddetails of the reserve requirements are summarised below.2.  Cash Reserve Ratio CRR2.1 Maintenance of CRRIn terms of Section 421 of the RBI Act 1934 Scheduled Commercial Banks are required tomaintain with RBI an average cash balance the amount of which shall not be less than three percent of the total of the Net Demand and Time Liabilities NDTL in India on a fortnightly basis andRBI is empowered to increase the said rate of CRR to such higher rate not exceeding twentypercent of the Net Demand and Time Liabilities NDTL under the R

To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.) [_tensor.py:575]


Generating short questions .........


Running model for generation
{'questions': [{'Question': "What is the RBI's statutory return for the year 2003?", 'Answer': 'india', 'id': 1, 'context': 'At   presenteffective from the fortnight beginning June 14 2003 the rate of CRR is 4.50 per cent of the NDTL.2.2 Maintenance of incremental CRRIn terms of Section 421A of RBI Act 1934 the Scheduled Commercial Banks are required tomaintain in addition to the balances prescribed under Section 421 of the Act an additionalaverage daily balance the amount of which shall not be less than the rate specified by the RBI inthe notification published in the Gazette of India such additional balance being calculated withreference to the excess of the total of the NDTL of the bank as shown in the return referred to insection 422 of the RBI Act 1934 over the total of its NDTL at the close of the business on thedate specified in the notification.At present no incremental CRR is required to be maintained by the S

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors


 Sense2vec_distractors successful for word :  india
context : correspondent bank under the remittance facilities scheme and remaining unpaid should bereflected in the accepting bank's books as an outside liability and the same should also be takeninto account for computation of NDTL for CRRSLR purpose.The amount received by correspondent banks has to be shown as 'Liability to the Banking System'by them and not as 'Liability to others' and this liability could be netted off by the correspondentbanks against the interbank assets. Likewise sums placed by banks issuingdraftsinterestdividend warrants are to be treated as 'Assets with Banking System' in their booksand can be netted off from their interbank liabilities.2.3.5 Other Demand and Time Liabilities ODTLOther Demand and Time Liabilities ODTL include interest accrued on deposits bills payableunpaid dividends suspense account balances representing amounts due to other banks or publicnet credit balances in branch adjustment account any 

...

 Sense2vec_distractors successful for word :  circulars
context : 413DBOD.No.BC5012.01.0012000017112000Collection of Data fromScheduled Commercial Banksin Annexure A and B2.3.814DBOD.No.Ret.BC.9912.01.00120020329042003Maintenance of Cash ReserveRatio CRR2.3.1015DBOD.No.BC.5412.01.00120020327122002Relaxation in Daily MinimumCash Reserve MaintenanceRequirement2.3.1116DBOD.No.BC.3412.01.00120010222102001Maintenance of Cash ReserveRatioCRR2.3.1217DBOD.No.BC.3412.01.00120010222102001Maintenance of Cash ReserveRatio CRR2.3.12118DBOD.Ret.BC.No.7912.01.001200220037032004Payment of interest on eligibleCRR balances on monthlybasisRevision in the format forsubmission of interest claimIntroduction of New software forForm A2.3.12 ii19DBOD.No.Ret.BC.9812.01.00120030418062004Revision of procedure forpayment of interest on theeligible CRR balances onmonthly basis2.3.12 ii20DBOD.No.Ret.BC.61C.96 Ret9024121990Shortfall in the maintenance ofCash Reserve Ratio CRRScheme of Graduated InterestRates2.3.12iii

Token indices sequence length is longer than the specified maximum sequence length for this model (1538 > 512). Running this sequence through the model will result in indexing errors


Running model for generation
{'questions': [{'Question': 'What is the name of the data collection system for the purpose of calculating the interest on the eligible CRR balances?', 'Answer': 'ret', 'id': 1, 'context': "413DBOD.No.BC5012.01.0012000017112000Collection of Data fromScheduled Commercial Banksin Annexure A and B2.3.814DBOD.No.Ret.BC.9912.01.00120020329042003Maintenance of Cash ReserveRatio CRR2.3.1015DBOD.No.BC.5412.01.00120020327122002Relaxation in Daily MinimumCash Reserve MaintenanceRequirement2.3.1116DBOD.No.BC.3412.01.00120010222102001Maintenance of Cash ReserveRatioCRR2.3.1217DBOD.No.BC.3412.01.00120010222102001Maintenance of Cash ReserveRatio CRR2.3.12118DBOD.Ret.BC.No.7912.01.001200220037032004Payment of interest on eligibleCRR balances on monthlybasisRevision in the format forsubmission of interest claimIntroduction of New software forForm A2.3.12 ii19DBOD.No.Ret.BC.9812.01.00120030418062004Revision of procedure forpayment of interest on theeligible CRR balances onm

Token indices sequence length is longer than the specified maximum sequence length for this model (1538 > 512). Running this sequence through the model will result in indexing errors


Running model for generation
 Sense2vec_distractors successful for word :  ret
Time taken to complete the entire task in seconds : 360.8422350883484


## Verifying the results

In [84]:
len(mcq_questions_list),len(short_questions_list),len(boolean_questions_list)

(12, 12, 12)

In [85]:
len(document_list)

12

In [86]:
df = pd.DataFrame({'Context':document_list,'mcq_question':mcq_questions_list,'short_question':short_questions_list,'boolean_question':boolean_questions_list})

In [87]:
df.head()

,Context,mcq_question,short_question,boolean_question
0,Master Circular Cash Reserve Ratio CRR and...,[What is the RBI's statutory return for the ye...,[What is the RBI's statutory return for the ye...,[Is the cash reserve ratio the same as the liq...
1,defined under Section 42 of RBI Act 1934 or to...,[What is the name of the Reserve Bank of India...,[What is the name of the Reserve Bank of India...,[Is there a difference between demand and time...
2,correspondent bank under the remittance facili...,[],[],[Is there a difference between other demand an...
3,c Amount received from DICGC towards claims an...,[What are the liabilities arising on account o...,[What are the liabilities arising on account o...,[Do commercial banks have to maintain statutor...
4,Loans out of Foreign Currency Non ŒResident Ac...,[],[],"[Is cash reserve ratio the same as crr?, Is th..."


In [92]:
import textwrap
wrapper = textwrap.TextWrapper(width=200)
word_list = wrapper.wrap(text=document_list[2])
for element in word_list:
  print(element)

correspondent bank under the remittance facilities scheme and remaining unpaid should bereflected in the accepting bank's books as an outside liability and the same should also be takeninto account
for computation of NDTL for CRRSLR purpose.The amount received by correspondent banks has to be shown as 'Liability to the Banking System'by them and not as 'Liability to others' and this liability
could be netted off by the correspondentbanks against the interbank assets. Likewise sums placed by banks issuingdraftsinterestdividend warrants are to be treated as 'Assets with Banking System' in
their booksand can be netted off from their interbank liabilities.2.3.5 Other Demand and Time Liabilities ODTLOther Demand and Time Liabilities ODTL include interest accrued on deposits bills
payableunpaid dividends suspense account balances representing amounts due to other banks or publicnet credit balances in branch adjustment account any amounts due to the Banking Systemwhich are not
in the nature o

In [93]:
word_list

["correspondent bank under the remittance facilities scheme and remaining unpaid should bereflected in the accepting bank's books as an outside liability and the same should also be takeninto account",
 "for computation of NDTL for CRRSLR purpose.The amount received by correspondent banks has to be shown as 'Liability to the Banking System'by them and not as 'Liability to others' and this liability",
 "could be netted off by the correspondentbanks against the interbank assets. Likewise sums placed by banks issuingdraftsinterestdividend warrants are to be treated as 'Assets with Banking System' in",
 'their booksand can be netted off from their interbank liabilities.2.3.5 Other Demand and Time Liabilities ODTLOther Demand and Time Liabilities ODTL include interest accrued on deposits bills',
 'payableunpaid dividends suspense account balances representing amounts due to other banks or publicnet credit balances in branch adjustment account any amounts due to the Banking Systemwhich are n

In [94]:
df.to_csv('questions.csv',index=False)